In [1]:
%load_ext autoreload
%autoreload 2
# -----------------------------------------------------------------------------------------------------------------------
# By Alexandra Lee
# (created Janurary 2019) 
#
# Encode Pseudomonas gene expression data into low dimensional latent space using 
# Tybalt with 2-hidden layers
# 
# Note: Need to use python 3 to support '*' syntax change
# --------------------------------------------------------------------------------------------------------------------
import os
import argparse
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
from functions.my_classes import DataGenerator
#from functions.vae_utils import VariationalLayer, WarmUpCallback, LossCallback
from functions.models import Tybalt

Using TensorFlow backend.


In [2]:
# To ensure reproducibility using Keras during development
# https://keras.io/getting-started/faq/#how-can-i-obtain-reproducible-results-using-keras-during-development
import numpy as np
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/keras-team/keras/issues/2280#issuecomment-306959926
randomState = 123
import os
os.environ['PYTHONHASHSEED'] = '0'

# The below is necessary for starting Numpy generated random numbers
# in a well-defined initial state.

np.random.seed(42)

# The below is necessary for starting core Python generated random numbers
# in a well-defined state.

rn.seed(12345)

# Force TensorFlow to use single thread.
# Multiple threads are a potential source of
# non-reproducible results.
# For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

from keras import backend as K

# The below tf.set_random_seed() will make random number generation
# in the TensorFlow backend have a well-defined initial state.
# For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

tf.set_random_seed(1234)

sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import keras
from keras.layers import Input, Dense, Lambda, Layer, Activation
from keras.layers.normalization import BatchNormalization
from keras.models import Model, Sequential
from keras import metrics, optimizers
from keras.callbacks import Callback

In [3]:
# --------------------------------------------------------------------------------------------------------------------
# Initialize hyper parameters
#
# learning rate: 
# batch size: Total number of training examples present in a single batch
#             Iterations is the number of batches needed to complete one epoch
# epochs: One Epoch is when an ENTIRE dataset is passed forward and backward through the neural network only ONCE
# kappa: warmup
# original dim: dimensions of the raw data
# latent dim: dimensiosn of the latent space (fixed by the user)
#   Note: intrinsic latent space dimension unknown
# epsilon std: 
# beta: Threshold value for ReLU?
# --------------------------------------------------------------------------------------------------------------------
learning_rate_model = 0.00001
epochs_model = 5
kappa_model = 0.01

intermediate_dim_model = 100
latent_dim_model = 10
epsilon_std_model = 1.0
beta_model = K.variable(0)

chunk_size = 100

# Get dimensions of datasets
train_dim_file =  os.path.join(os.path.dirname(os.getcwd()), "metadata", "train_dim.pickle")
val_dim_file =  os.path.join(os.path.dirname(os.getcwd()), "metadata", "validation_dim.pickle")

with open(train_dim_file, 'rb') as f:
    num_train_samples, num_genes = pickle.load(f)
with open(val_dim_file, 'rb') as f:
    num_val_samples, num_genes = pickle.load(f)

original_dim_model = num_genes
print(num_train_samples)#num_samples_train = 1319122
print(num_val_samples)#num_samples_val = 1319

1317819
1319


In [4]:
# Load gene expression data using generator 
train_file =  "/home/alexandra/Documents/Data/LINCS_tuning/train_model_input.txt.xz"
validation_file = "/home/alexandra/Documents/Data/LINCS_tuning/validation_model_input.txt.xz"

training_generator = DataGenerator(train_file, chunk_size, num_train_samples)
validation_generator = DataGenerator(validation_file, chunk_size, num_val_samples)

In [5]:
# Model 
model = Tybalt(original_dim=original_dim_model,
                 hidden_dim=intermediate_dim_model,
                 latent_dim=latent_dim_model,
                 batch_size=chunk_size,
                 epochs=epochs_model,
                 learning_rate=learning_rate_model,
                 kappa=kappa_model,
                 beta=beta_model,
                 epsilon_std=epsilon_std_model)

In [6]:
# Output files
stat_file =  os.path.join(os.path.dirname(os.getcwd()), "stats", "tybalt_2layer_{}latent_stats.tsv".format(latent_dim_model))
hist_plot_file = os.path.join(os.path.dirname(os.getcwd()), "stats", "tybalt_2layer_{}latent_hist.png".format(latent_dim_model))

encoded_file = os.path.join(os.path.dirname(os.getcwd()), "encoded", "train_input_2layer_{}latent_encoded.txt".format(latent_dim_model))

model_encoder_file = os.path.join(os.path.dirname(os.getcwd()), "models", "tybalt_2layer_{}latent_encoder_model.h5".format(latent_dim_model))
weights_encoder_file = os.path.join(os.path.dirname(os.getcwd()), "models", "tybalt_2layer_{}latent_encoder_weights.h5".format(latent_dim_model))
model_decoder_file = os.path.join(os.path.dirname(os.getcwd()), "models", "tybalt_2layer_{}latent_decoder_model.h5".format(latent_dim_model))
weights_decoder_file = os.path.join(os.path.dirname(os.getcwd()), "models", "tybalt_2layer_{}latent_decoder_weights.h5".format(latent_dim_model))

In [7]:
# Compile Model
model.build_encoder_layer()
model.build_decoder_layer()
model.compile_vae()
model.get_summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 978)          0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 100)          97900       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 100)          400         dense_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 100)          0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
dense_2 (D

/home/alexandra/Documents/Repos/LINCS_latent_space/scripts/functions/models.py:85: UserWarning: Output "custom_variational_layer_1" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "custom_variational_layer_1" during training.
  self.vae.compile(optimizer=adam, loss=None, loss_weights=[self.beta])


In [8]:
# Model architecture
model_architecture_file = os.path.join(os.path.dirname(os.getcwd()),'stats', 'vae_architecture.png')
model.visualize_architecture(model_architecture_file)

In [9]:
%%time
model.train_vae(train_df=training_generator,
                test_df=validation_generator,
               separate_loss=True)

updating callback loss...


AttributeError: 'Tybalt' object has no attribute 'encoder'

In [10]:
model_training_file = os.path.join(os.path.dirname(os.getcwd()), 'stats', 'training.pdf')
model.visualize_training(model_training_file)

AttributeError: 'Tybalt' object has no attribute 'hist'

In [ ]:
model_compression = model.compress(rnaseq_df)
model_file = os.path.join(os.path.dirname(os.getcwd()), 'data', 'encoded_rnaseq_twohidden_100model.tsv.gz')
model_compression.to_csv(model_file, sep='\t', compression='gzip')
model_compression.head(2)

In [ ]:
model_weights = model.get_decoder_weights()

In [ ]:
encoder_model_file = os.path.join(os.path.dirname(os.getcwd()),'models', 'encoder_twohidden100_vae.hdf5')
decoder_model_file = os.path.join(os.path.dirname(os.getcwd()),'models', 'decoder_twohidden100_vae.hdf5')
model.save_models(encoder_model_file, decoder_model_file)